In [7]:
#Import dependencies
import pandas as pd
import sqlite3 as sqlite
from datetime import datetime as dt
import numpy as np

#Path name files to datasets for cleaning and processing.  Jeopardy CSV is from Kaggle, Contestants and Locations
# are from github/anuparna/jeopardy.  jeaopardy_archive is the flat file Geetha found
path = ('./Resources/jeopardy_archive_data.csv')
path2 = ('./Resources/JEOPARDY_CSV.csv')
path3 = ('./Resources/contestants.csv')
path4 = ('./Resources/locations.csv')
path5 = ('./Resources/final_results.csv')


In [8]:
#Connector to sqlite3.  Creates Jeopardy database
conn = sqlite.connect('jeopardy_db.sqlite')

In [9]:
from numpy import datetime64

#Read and format csv files into dataframe.  
#Renamed the columns in 'JEOPARDY_CSV.csv' to find fields to join the two datasets.

df = pd.read_csv(path)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)
df4 = pd.read_csv(path4)
df5 = pd.read_csv(path5)

jeopardy_df1 = pd.DataFrame(df)
jeopardy_df1['date'] = pd.to_datetime(jeopardy_df1['date'], errors='coerce')
jeopardy_df1['date'] = jeopardy_df1['date'].dt.strftime('%m/%d/%Y')
jeopardy_df1 = pd.DataFrame(jeopardy_df1).rename(columns={'full_name': 'Name'})

#Removed extra text from jeopardy_archive data to join fields on "show_info"

jeopardy_df1['show_info'] = jeopardy_df1['show_info'].str.replace(r'\D','')
jeopardy_df1['show_info'] = jeopardy_df1['show_info'].astype('int')
num=100000000
jeopardy_df1['show_info'] =(jeopardy_df1['show_info']/num).round(0)
jeopardy_df1['show_info'] = jeopardy_df1['show_info'].astype('int')


jeopardy_df2 = pd.DataFrame(df2).rename(columns={'Show Number': 'show_info', ' Air Date': 'date', ' Round': 'Round_', ' Category': 'Category', ' Value': 'Value',
' Question': 'Question', ' Answer': 'Answer'})
jeopardy_df2['date'] = pd.to_datetime(jeopardy_df2['date'], errors='coerce')
jeopardy_df2['date'] = jeopardy_df2['date'].dt.strftime('%m/%d/%Y')

jeopardy_df3 = pd.DataFrame(df3)
jeopardy_df3['Name'] = jeopardy_df3['player_first_name'].map(str) + ' ' + jeopardy_df3['player_last_name'].map(str)
jeopardy_df3 = jeopardy_df3.drop(columns = ['player_first_name', 'player_last_name'])
jeopardy_df3 = jeopardy_df3.drop_duplicates(subset = 'Name')

jeopardy_df4 = pd.DataFrame(df4)
jeopardy_df5 = pd.DataFrame(df5)



/Users/Justin/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version.


In [10]:
#Read datasets into Sqlite table

jeopardy_df1.to_sql('jeopardy_archvie_tbl', conn, if_exists='replace', index = False)
jeopardy_df2.to_sql('jeopardy_kaggle_tbl', conn, if_exists='replace', index = False)
jeopardy_df3.to_sql('jeopardy_contestants_tbl', conn, if_exists='replace', index = False)
jeopardy_df4.to_sql('jeopardy_seat_locations_tbl', conn, if_exists='replace', index = False)
jeopardy_df5.to_sql('jeopardy_final_results_tbl', conn, if_exists='replace', index = False)
conn.commit()


In [11]:
#Check headers to use in SQL queries
print(f'jeopardy_archive:', list(jeopardy_df1.columns))
print(f' JEOPARDY_CSV:', list(jeopardy_df2.columns))
print(f' contestants:',list(jeopardy_df3.columns))
print(f' locations', list(jeopardy_df4.columns))
print(f' final_results', list(jeopardy_df5.columns))

jeopardy_archive: ['nickname', 'player_details', 'final_score', 'show_info', 'Name', 'occupation', 'hometown', 'archive_info', 'date']
 JEOPARDY_CSV: ['show_info', 'date', 'Round_', 'Category', 'Value', 'Question', 'Answer']
 contestants: ['player_id', 'hometown_city', 'hometown_state', 'occupation', 'Name']
 locations ['game_id', 'player_id', 'seat_location', 'season']
 final_results ['game_id', 'season', 'position', 'dj_score', 'wager', 'correct', 'coryat_score']


In [12]:
SQL_join1 = pd.read_sql(' \
SELECT B.Name, A.seat_location, C.correct, D.show_info  \
FROM jeopardy_seat_locations_tbl A \
LEFT JOIN jeopardy_contestants_tbl B ON A.player_id = B.player_id \
INNER JOIN jeopardy_final_results_tbl C ON C.game_id = A.game_id \
INNER JOIN jeopardy_archvie_tbl D ON B.Name = D.Name'
                        
                        
                        
                        
                        , conn)  

SQL_join1.to_sql('jeopardy_sql_query1', conn, if_exists='replace', index = False)
conn.commit()

In [13]:
SQL_join2 = pd.read_sql(' \
SELECT Q.Name, Q.seat_location, Q.correct, Q.show_info, A. Category, A.question, A.Answer, A.value  \
FROM jeopardy_sql_query1 Q \
LEFT JOIN jeopardy_kaggle_tbl A ON Q.show_info = A.show_info', conn)

SQL_join2.dropna().sort_values(by= 'show_info').to_sql('jeopardy_sql_query2', conn, if_exists='replace', index = False)
conn.commit()
conn.close()

